In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
from __future__ import print_function
from util_functions import process_files

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [40]:
batch_size = 64
num_classes = 2
epochs = 100

# load the data
x_train, y_train, f_train = process_files(
                                    dataset='training', 
                                    features=['Mel'], 
                                    shape='flat')

x_test, y_test, f_test = process_files(
                                    dataset='test', 
                                    features=['Mel'], 
                                    shape='flat')

import numpy as np
from numpy import linalg as LA

#normalise
x_train = x_train - x_train.mean()
x_test = x_test - x_test.mean()
x_train = x_train/LA.norm(x_train)
x_test = x_test/LA.norm(x_test)

# input image dimensions
input_d = x_train.shape[1] #Depth
input_h = x_train.shape[2] #Height
input_w = x_train.shape[3] #Width

#Reshaping to feed to network
x_train = x_train.reshape(x_train.shape[0], input_h, input_w, input_d)
x_test = x_test.reshape(x_test.shape[0], input_h, input_w, input_d)
input_shape = (input_h, input_w, input_d)

#Making them floats for TF
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# iterate over different hyperparameters to finetune them
# from the results in 10 epochs we can see that the other learning rates are just too big. 
# Hence only this one will be used
learning_rate = 0.0001
dropout_rates = [0.25]

for dr in dropout_rates:

    print('learning rate fixed at 0.0001, dropout: ' + str(dr) + '\n')        
    model = Sequential()
    model.add(Conv2D(80, kernel_size=(57, 6),
                     activation='relu',
                     input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(80, kernel_size=(1, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(8, 7), strides=(8, 7)))
    model.add(Dropout(dr))
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(dr))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(dr))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.binary_crossentropy,
                  optimizer=keras.optimizers.Nadam(lr=learning_rate),
                  metrics=['accuracy'])

    tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0,  
          write_graph=True, write_images=True)
    
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=2,
              validation_split=0.1,
              callbacks=[tbCallBack])
              
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])        

    print('----------------------------------------------------\n')    

learning rate fixed at 0.0001, dropout: 0.25

Train on 1306 samples, validate on 146 samples
Epoch 1/100
 - 2s - loss: 0.6723 - acc: 0.7619 - val_loss: 0.8495 - val_acc: 0.0000e+00
Epoch 2/100
 - 1s - loss: 0.5673 - acc: 0.7741 - val_loss: 1.4704 - val_acc: 0.0000e+00
Epoch 3/100
 - 1s - loss: 0.5357 - acc: 0.7741 - val_loss: 1.4306 - val_acc: 0.0000e+00
Epoch 4/100
 - 1s - loss: 0.5351 - acc: 0.7741 - val_loss: 1.3604 - val_acc: 0.0000e+00
Epoch 5/100
 - 1s - loss: 0.5359 - acc: 0.7741 - val_loss: 1.4911 - val_acc: 0.0000e+00
Epoch 6/100
 - 1s - loss: 0.5326 - acc: 0.7741 - val_loss: 1.4340 - val_acc: 0.0000e+00
Epoch 7/100
 - 1s - loss: 0.5310 - acc: 0.7741 - val_loss: 1.4050 - val_acc: 0.0000e+00
Epoch 8/100
 - 1s - loss: 0.5286 - acc: 0.7741 - val_loss: 1.3791 - val_acc: 0.0000e+00
Epoch 9/100
 - 1s - loss: 0.5228 - acc: 0.7741 - val_loss: 1.5455 - val_acc: 0.0000e+00
Epoch 10/100
 - 1s - loss: 0.5117 - acc: 0.7741 - val_loss: 1.4201 - val_acc: 0.0068
Epoch 11/100
 - 1s - loss: 0.4

Epoch 96/100
 - 1s - loss: 0.3159 - acc: 0.8652 - val_loss: 1.2151 - val_acc: 0.6164
Epoch 97/100
 - 1s - loss: 0.3240 - acc: 0.8476 - val_loss: 0.9040 - val_acc: 0.6233
Epoch 98/100
 - 1s - loss: 0.3215 - acc: 0.8622 - val_loss: 1.0545 - val_acc: 0.6233
Epoch 99/100
 - 1s - loss: 0.3191 - acc: 0.8691 - val_loss: 1.0729 - val_acc: 0.6164
Epoch 100/100
 - 1s - loss: 0.3234 - acc: 0.8576 - val_loss: 1.1874 - val_acc: 0.6164
Test loss: 0.58175981561274
Test accuracy: 0.8024263431542461
----------------------------------------------------

